In [ ]:
import urllib
import numpy as np
import tensorflow as tf
import csv

# DO NOT CHANGE THIS CODE
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[1:]))
    return ds.batch(batch_size).prefetch(1)


def solution_model():
    url = 'https://storage.googleapis.com/download.tensorflow.org/data/Sunspots.csv'
    urllib.request.urlretrieve(url, 'sunspots.csv')

    time_step = []
    sunspots = []

    with open('sunspots.csv') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        next(reader)
        for row in reader:
            sunspots.append(float(row[2]))
            time_step.append(int(row[0]))

    series = np.array(sunspots)

    # DO NOT CHANGE THIS CODE
    # This is the normalization function
    min = np.min(series)
    max = np.max(series)
    series -= min
    series /= max
    time = np.array(time_step)

    # The data should be split into training and validation sets at time step 3000
    # DO NOT CHANGE THIS CODE
    split_time = 3000

    time_train = time[:split_time]  # YOUR CODE HERE
    x_train = series[:split_time]  # YOUR CODE HERE
    time_valid = time[split_time:]  # YOUR CODE HERE
    x_valid = series[split_time:]  # YOUR CODE HERE

    # DO NOT CHANGE THIS CODE
    window_size = 30
    batch_size = 32
    shuffle_buffer_size = 1000

    train_set = windowed_dataset(x_train, window_size=window_size, batch_size=batch_size,
                                 shuffle_buffer=shuffle_buffer_size)

    model = tf.keras.models.Sequential([
        # YOUR CODE HERE. Whatever your first layer is, the input shape will be [None,1] when using the Windowed_dataset above, depending on the layer type chosen
        tf.keras.layers.Conv1D(filters=32, kernel_size=5, strides=1, padding='causal', activation='relu', input_shape=[None,1]),
        tf.keras.layers.LSTM(60, return_sequences=True),
        tf.keras.layers.LSTM(60, return_sequences=True),
        tf.keras.layers.Dense(30, activation="relu"),
        tf.keras.layers.Dense(10, activation="relu"),
        tf.keras.layers.Dense(1)
    ])
    # PLEASE NOTE IF YOU SEE THIS TEXT WHILE TRAINING -- IT IS SAFE TO IGNORE
    # BaseCollectiveExecutor::StartAbort Out of range: End of sequence
    # 	 [[{{node IteratorGetNext}}]]
    #

    # YOUR CODE HERE TO COMPILE AND TRAIN THE MODEL
    class myCallback(tf.keras.callbacks.Callback):
      def on_epoch_end(self, epoch, logs={}):
        if(logs.get('mae')<=0.12):
          print("\nMae of 0.12 so cancelling training!")
          self.model.stop_training = True

    callbacks = myCallback()

    optimizer = tf.keras.optimizers.SGD(lr=1e-5, momentum=0.9)
    model.compile(loss=tf.keras.losses.Huber(),
                  optimizer=optimizer,
                  metrics=["mae"])
    model.fit(train_set, epochs=500, callbacks=[callbacks])
    return model


# Note that you'll need to save your model as a .h5 like this.
# When you press the Submit and Test button, your saved .h5 model will
# be sent to the testing infrastructure for scoring
# and the score will be returned to you.
if __name__ == '__main__':
    model = solution_model()
    model.save("mymodel.h5")

Epoch 1/500
93/93 [==============================] - 1s 9ms/step - loss: 0.0398 - mae: 0.2178
Epoch 2/500
93/93 [==============================] - 1s 8ms/step - loss: 0.0385 - mae: 0.2127
Epoch 3/500
93/93 [==============================] - 1s 9ms/step - loss: 0.0375 - mae: 0.2087
Epoch 4/500
93/93 [==============================] - 1s 9ms/step - loss: 0.0366 - mae: 0.2056
Epoch 5/500
93/93 [==============================] - 1s 9ms/step - loss: 0.0359 - mae: 0.2030
Epoch 6/500
93/93 [==============================] - 1s 9ms/step - loss: 0.0353 - mae: 0.2007
Epoch 7/500
93/93 [==============================] - 1s 9ms/step - loss: 0.0347 - mae: 0.1987
Epoch 8/500
93/93 [==============================] - 1s 9ms/step - loss: 0.0342 - mae: 0.1968
Epoch 9/500
93/93 [==============================] - 1s 9ms/step - loss: 0.0338 - mae: 0.1950
Epoch 10/500
93/93 [==============================] - 1s 8ms/step - loss: 0.0334 - mae: 0.1933
Epoch 11/500
93/93 [==============================] - 1s 9m